# Glasswall CDR Folder-to-Folder Workflows

In [24]:
import ipywidgets as widgets
from osbot_utils.utils.Http import GET_json
from osbot_utils.utils.Json import json_to_str,str_to_json
from jupyter_apis.API_Client import API_Client

DEFAULT_BASE_DIR = 'scenario-1'
DEFAULT_THREADS  = '25'

class F2f_Workflows:
    def __init__(self):
        self.output       = None
        self.buttons      = None
        self.text_target  = None
        self.text_threads = None
        self.api          = None
        
    def add_button(self, text, style='success',icon='check'):
        button = widgets.Button(
            description  = text   ,
            disabled     = False  ,
            button_style = style  , # 'success', 'info', 'warning', 'danger' or ''
            tooltip      = text   ,
            icon         = icon   # (FontAwesome names without the `fa-` prefix)
        )
        button.on_click(self.button_clicked)
        return button

    def set_output(self):
        self.output_text = widgets.Textarea(layout={'height': '100%','width':'100%'})
        self.output_box  = widgets.VBox([self.output_text], layout={'height': '130px', 'width':'100%'})
        return self

    def setup(self):
        self.api = API_Client()
        self.set_output()
        return self
    
    def get_buttons(self): 
        button_clear = self.add_button('clear data'      , 'danger' )
        button_load = self.add_button('load files'       , 'info'   )
        button_start = self.add_button('start processing', 'success')
        button_stop = self.add_button('stop processing'  , 'warning')
        self.buttons = [button_start,button_stop, button_load, button_clear]
        return widgets.HBox(self.buttons)
    
    def get_set_target(self):
        self.text_target   = widgets.Text(value=DEFAULT_BASE_DIR, layout={'width':'200px'})
        self.text_threads  = widgets.Text(value=DEFAULT_THREADS, layout={'width':'100px'})  
        label_threads = widgets.Label("Threads:")
        button_target = widgets.Button(description='Set base dir')
        button_target.on_click(self.button_clicked)
        return widgets.HBox([ button_target, self.text_target, label_threads, self.text_threads])
            
    def add_output(self, message):     
        f2f_workflows.output_text.value = message + '\n' + f2f_workflows.output_text.value 
        #f2f_workflows.output_text.value += message + '\n' 

    def button_clicked(self,source):
        #self.add_output(f'button clicked: {source.description}')
        if source.description == 'load files':
            self.add_output('... starting loading files workflow')
            try:
                self.add_output(self.api.pre_process())
                self.add_output('... all files loaded')
            except Exception as error:
                self.add_output(f"Error: {error}")
        elif source.description == 'clear data':
            try:
                self.add_output(str_to_json(self.api.clear_data_and_status()).get('message'))
                self.add_output(self.api.reload_elastic_file_metadata())
            except Exception as error:
                self.add_output(f"Error: {error}")
        elif source.description == 'start processing':
            thread_count = self.text_threads.value
            self.add_output(f'... starting processing with {thread_count} threads') 
            try: 
                self.add_output(str_to_json(self.api.start_process(thread_count)))
            except Exception as error:
                self.add_output(f"Error: {error}")                        
        elif source.description == 'stop processing':
            self.add_output('... stopping processing')
            try:
                self.add_output(str_to_json(self.api.stop_process()))
            except Exception as error:
                self.add_output(f"Error: {error}")            
        elif source.description == 'Set base dir':
            try:                        
                base_dir = self.text_target.value
                data = {
                          "hd1_path": f"./test_data/{base_dir}/hd1",
                          "hd2_path": f"./test_data/{base_dir}/hd2",
                          "hd3_path": f"./test_data/{base_dir}/hd3"
                        }
                self.add_output(f'Setting target folders to {json_to_str(data)}')
                self.add_output("result:  \n" + self.api.configure_environment(data).text + "/n")
            except Exception as error:
                self.add_output(f"Error: {error}")            
            
        else:                
            self.add_output(f'unrecognised command:  {source.description}')

In [25]:
f2f_workflows = F2f_Workflows().setup()

In [26]:
f2f_workflows.get_set_target()

TraitError: The 'value' trait of a Text instance expected a unicode string, not the int 25.

In [23]:
buttons = f2f_workflows.get_buttons()
buttons

In [21]:
print("output\n")
f2f_workflows.output_box

output



In [27]:
#f2f_workflows.buttons[0].click()
#f2f_workflows.buttons[1].click()
#f2f_workflows.buttons[2].click() 